In [1]:
# Python SQL toolkit and Object Relational Mapper
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [4]:
#Create Measurement class

Base = declarative_base()

class Measurement(Base):
    
    __tablename__ = 'hawaii_measurements'

    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Float)
    
class Stations(Base):
    __tablename__ = 'hawaii_stations'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    #Is this needed? In practice codes...
#     def __repr__(self):
#         return f"id={self.id}, name={self.station}"

In [5]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [6]:
# Load the cleaned csv file into a pandas dataframe
new_hawaii_measurements = pd.read_csv('Resources/clean_hawaii_measurements.csv')
# Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata 
measurement_data = new_hawaii_measurements.to_dict(orient='records')
print(measurement_data[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]


In [7]:
new_hawaii_stations=pd.read_csv('Resources/clean_hawaii_stations.csv')

station_data = new_hawaii_stations.to_dict(orient='records')
print(station_data[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [8]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [9]:
# Save the reference to the measurements and stations table as variables
measurements_table = sqlalchemy.Table('hawaii_measurements', metadata, autoload=True)
stations_table = sqlalchemy.Table('hawaii_stations', metadata, autoload=True)

In [10]:
# Use `table.delete()` to remove any pre-existing data.
# convenience function so that you can re-run the example code multiple times.
conn.execute(measurements_table.delete())
conn.execute(stations_table.delete())

In [11]:
# `table.insert()` to insert the data into the table; The SQL table is populated during this step
conn.execute(measurements_table.insert(), measurement_data)
conn.execute(stations_table.insert(), station_data)

In [12]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [13]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]